In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

import warnings
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import xgboost as xgb


import eli5
from eli5.sklearn import PermutationImportance
from eli5.permutation_importance import get_score_importances

/Users/morphine/.pyenv/versions/anaconda3-5.3.1/envs/py38/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/morphine/.pyenv/versions/anaconda3-5.3.1/envs/py38/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
import pandas as pd # 데이터 분석
import numpy as np # 행렬 연산, version: 1.6.1

import os
import sys

import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
import sklearn
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD, PCA, FastICA, FactorAnalysis, KernelPCA, DictionaryLearning
from sklearn.decomposition import IncrementalPCA, LatentDirichletAllocation,MiniBatchSparsePCA, SparsePCA

import itertools

In [4]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
sns.set(font_scale=2.5)

warnings.filterwarnings('ignore')

# 데이터 로드

In [5]:
data_dir = Path('../input/dankook')
sub_dir = Path('../output/')

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'
sub_file = sub_dir / 'submission.csv'

target_column = 'class'
SEED = 2020

In [6]:
df_train = pd.read_csv(train_file, index_col=0)
df_test = pd.read_csv(test_file, index_col=0)

# 이상치 제거

In [7]:
# test의 MinMax 범위 넘는 행은 train에서 제거
train_shape = df_train.shape[0]

for col in df_train.columns[:18]:
    df_train = df_train.loc[np.logical_and(df_train[col]>=df_test[col].min(),
                            df_train[col]<=df_test[col].max())]

print('제거된 행 개수 :', train_shape - df_train.shape[0])

제거된 행 개수 : 77


In [8]:
df_train.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
count,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,3.1992e+05,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000
mean,19.8525,18.4318,17.7352,17.3037,17.0603,6.0481e-02,19.6298,18.2591,17.6158,17.2151,16.9944,6.3334,6.1396,1.1758,1.1765,1.1751,1.1754,1.1761,1.1165
std,1.9392,1.6598,1.4586,1.3141,1.3310,2.9110e-01,1.9114,1.6594,1.4672,1.3250,1.3232,8.8817,8.5522,0.1163,0.1181,0.1147,0.1155,0.1171,0.9234
min,2.2651,-12.4441,7.7314,7.7115,-9.2548,-2.5915e+01,-30.6337,-18.6560,-8.7560,-3.6973,0.2159,1.0000,1.0000,1.0001,1.0001,1.0002,1.0002,1.0002,0.0000
25%,18.7244,17.4750,16.8773,16.5240,16.2890,3.5724e-05,18.5639,17.3486,16.7874,16.4532,16.2343,1.0000,1.0000,1.0883,1.0885,1.0878,1.0881,1.0883,0.0000
50%,19.4195,18.1405,17.5259,17.1498,16.9177,4.7153e-02,19.2647,18.0225,17.4434,17.0874,16.8694,2.0000,2.0000,1.1794,1.1792,1.1794,1.1794,1.1793,1.0000
75%,20.4320,19.0728,18.4279,18.0074,17.7288,9.4606e-02,20.1976,18.8883,18.2908,17.9070,17.6555,5.0000,5.0000,1.2275,1.2260,1.2292,1.2286,1.2268,2.0000
max,49.1436,46.3383,45.1299,32.8634,52.6127,4.4620e+01,30.7779,30.6132,31.2940,30.5509,28.5710,44.0000,42.0000,2.0491,2.0786,2.0205,2.0347,2.0637,2.0000


In [9]:
df_test.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z
count,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,8.0000e+04,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000
mean,19.8598,18.4371,17.7397,17.3065,17.1490,6.0083e-02,19.6350,18.2638,17.6193,17.2178,16.8702,6.3499,6.1578,1.1760,1.1767,1.1753,1.1756,1.1763
std,1.9543,1.7030,1.4786,1.3207,24.6431,3.4684e-01,1.9210,1.6657,1.4760,1.3324,35.4367,8.8728,8.5509,0.1164,0.1182,0.1148,0.1156,0.1173
min,-0.3385,-51.1753,-5.4387,5.3267,-39.5272,-3.0149e+01,-30.6337,-18.6560,-8.7560,-3.6973,-9999.0000,1.0000,1.0000,1.0001,1.0001,1.0002,1.0002,1.0001
25%,18.7273,17.4780,16.8820,16.5255,16.2882,3.3645e-05,18.5709,17.3522,16.7906,16.4562,16.2310,1.0000,1.0000,1.0881,1.0883,1.0877,1.0879,1.0882
50%,19.4261,18.1451,17.5299,17.1560,16.9212,4.7115e-02,19.2674,18.0247,17.4487,17.0912,16.8733,2.0000,2.0000,1.1794,1.1793,1.1794,1.1794,1.1793
75%,20.4344,19.0793,18.4345,18.0080,17.7333,9.4769e-02,20.1999,18.8948,18.2952,17.9109,17.6578,5.0000,5.0000,1.2278,1.2262,1.2294,1.2289,1.2269
max,56.8471,94.3591,46.6913,33.0259,6976.3922,4.6390e+01,30.8899,30.9529,31.6536,30.9478,28.6441,44.0000,42.0000,2.0502,2.0797,2.0216,2.0358,2.0648


# 변수 생성 

In [10]:
for i in ['u','g','r','i','z']:
    df_train[f'ugriz_{i}'] = df_train[i]
    df_test[f'ugriz_{i}'] = df_test[i]
df_train.drop(columns=['u','g','r','i','z'], axis=1, inplace=True)
df_test.drop(columns=['u','g','r','i','z'], axis=1, inplace=True)

In [11]:
ugriz_col = [c for c in df_train.columns if c.find('ugriz') != -1]
dered_col = [c for c in df_train.columns if c.find('dered') != -1]
airmass_col = [c for c in df_train.columns if c.find('airmass') != -1]

In [12]:
# zip 함수를 이용하여 각 Row별, Magnitude별, max, min, max-min, std, sum을 구한다.
for prefix , g in zip(['ugriz','dered','airmass'], [ugriz_col, dered_col, airmass_col]):
    df_train[f'{prefix}_max'] = df_train[g].max(axis=1)
    df_test[f'{prefix}_max'] = df_test[g].max(axis=1)
    
    df_train[f'{prefix}_min'] = df_train[g].min(axis=1)
    df_test[f'{prefix}_min'] = df_test[g].min(axis=1)
    
    df_train[f'{prefix}_diff'] = df_train[f'{prefix}_max'] - df_train[f'{prefix}_min']
    df_test[f'{prefix}_diff'] = df_test[f'{prefix}_max'] - df_test[f'{prefix}_min']
    
    df_train[f'{prefix}_std'] = df_train[g].std(axis=1)
    df_test[f'{prefix}_std'] = df_test[g].std(axis=1)
    
    df_train[f'{prefix}_sum'] = df_train[g].sum(axis=1)
    df_test[f'{prefix}_sum'] = df_test[g].sum(axis=1)

In [13]:
# diff feature 추가 예: z - i
# itertools combinations을 활용하여 전체 magnitude에서 diff를 구함
# 총 105가지 조합이 나옴. 여기서 안 좋은 것은 permutation importance를 활용하여 제거할 예정
diff_feature = []
for c1, c2 in itertools.combinations(ugriz_col[::-1]+dered_col[::-1]
                                     +airmass_col[::-1],2):
    new_c = f'{c1}_{c2}_diff'
    df_train[new_c] = df_train[c1]-df_train[c2]
    df_test[new_c] = df_test[c1]-df_test[c2]
    diff_feature.append(new_c)
    

In [14]:
# 각 maginitude 별 max-max, min-min, sum-sum 을 구함

for c in itertools.combinations(['ugriz','dered','airmass'],2):
    df_train[f'{c[0]}_{c[1]}_max_diff'] = df_train[f'{c[0]}_max'] - df_train[f'{c[1]}_max']
    df_test[f'{c[0]}_{c[1]}_max_diff'] = df_test[f'{c[0]}_max'] - df_test[f'{c[1]}_max']
    
    df_train[f'{c[0]}_{c[1]}_min_diff'] = df_train[f'{c[0]}_min'] - df_train[f'{c[1]}_min']
    df_test[f'{c[0]}_{c[1]}_min_diff'] = df_test[f'{c[0]}_min'] - df_test[f'{c[1]}_min']
    
    df_train[f'{c[0]}_{c[1]}_sum_diff'] = df_train[f'{c[0]}_sum'] - df_train[f'{c[1]}_sum']
    df_test[f'{c[0]}_{c[1]}_sum_diff'] = df_test[f'{c[0]}_sum'] - df_test[f'{c[1]}_sum']

In [15]:
# 아래와 같은 공식들을 구현함
# http://classic.sdss.org/dr4/algorithms/sdssUBVRITransform.html 

def make_2flux_feature(train, test, c1, c2, func, mag_list=None):
    
    for c in mag_list:
        x=train[f'{c}_{c1}'].values
        y=train[f'{c}_{c2}'].values
        train[f'{c}_{func.__name__}'] = func(x,y)
        
        x=test[f'{c}_{c1}'].values
        y=test[f'{c}_{c2}'].values
        test[f'{c}_{func.__name__}'] = func(x,y)
        
def UB_jester(x1,x2):
    return 0.75*(x1-x2)-0.81

make_2flux_feature(df_train,df_test, 'u','g',UB_jester,['ugriz'])

def BV_jester(x1,x2):
    return 0.62*(x1-x2)+0.15

make_2flux_feature(df_train,df_test, 'g','r',BV_jester,['ugriz'])

def VR_jester(x1,x2):
    return 0.38*(x1-x2)+0.27

make_2flux_feature(df_train, df_test, 'r','i',VR_jester,['ugriz'])

def RcIc_jester(x1,x2):
    return 0.72*(x1-x2)+0.27

make_2flux_feature(df_train, df_test, 'r','i', RcIc_jester, ['ugriz'])

def B_jester(x1,x2):
    return x2+0.17*(x1-x2)+0.11

make_2flux_feature(df_train,df_test, 'u','g', B_jester, ['ugriz'])

def V_jester(x1,x2):
    return x1-0.52*(x1-x2)-0.03

make_2flux_feature(df_train,df_test, 'g','r',V_jester, ['ugriz'])

In [16]:
# https://www.sdss.org/dr16/algorithms/segue_target_selection/#Legacy

# 아래 나열된 모든 ugriz 등급은 달리 명시되지 않는 한 dereddened PSF mag입니다.
# dered_X 사용하면 될듯, 일단은 잘 몰라서 ugriz_X 버전과 dered_X 버전을 비교해서 사용하면 될듯.

# ugriz_X 버전

all_data = pd.concat([df_train,df_test], axis=0)
all_data.fillna(-1, inplace=True)

all_data['ugriz_icolor'] = all_data['ugriz_u']*(-0.436) + all_data['ugriz_g']*(1.129) + all_data['ugriz_r']*(-0.119) + all_data['ugriz_i']*(-0.574) + 0.1984

all_data['ugriz_ucolor'] = all_data['ugriz_u']*(-0.249) + all_data['ugriz_g']*(0.794) + all_data['ugriz_r']*(-0.555) + 0.234

all_data['ugriz_p1'] = (all_data['ugriz_u']-all_data['ugriz_g'])*(0.91) + (all_data['ugriz_g']-all_data['ugriz_r'])*(0.415)- 1.280

all_data['ugriz_r_std_div'] = all_data['ugriz_r']/all_data['ugriz_r'].std()

In [17]:
# https://www.sdss.org/dr16/algorithms/legacy_target_selection/
# 멸망과 관련 있는 거라고 함.
# dered_X 가 어떤 곳에서는 멸망과 관련 있는 데이터였음.
# ugriz_X를 붉어짐에 대한 걸로 사용하다면, 여기서는 dered_X를 멸망과 관련 있는 것과 사용하면 될듯

all_data['dered_orthogonal'] = (all_data['dered_r']-all_data['dered_i'])-(all_data['dered_g']-all_data['dered_r'])/4-0.18

all_data['dered_parallel'] = 0.7*(all_data['dered_g']-all_data['dered_r']) + 1.2*((all_data['dered_r']-all_data['dered_i'])-0.18)

In [18]:
# https://www.sdss.org/dr12/algorithms/magnitudes/
# 문서에는 psfMag에 대한 데이터인데, 그것을 나는 ugriz로 사용하고 있으니까 urgiz로 사용

color_list = ['u', 'g', 'r', 'i', 'z']
b_list = [1.4*10e-10, 0.9*10e-10, 1.2*10e-10, 1.8*10e-10, 7.4*10e-10]
f0_list = [24.63, 25.11, 24.80, 24.36, 22.83]
for c, b, f0 in zip(color_list, b_list, f0_list):
    all_data[f'ugriz_{c}_asinh'] = -2.5*np.log(10)*(np.arcsinh((all_data[f'ugriz_{c}']/f0)/(2*b))+np.log(b))

# feature 선택

In [19]:
selected_columns = ['redshift','dered_r', 'dered_i', 'airmass_g', 'ugriz_r', 'ugriz_i', 'airmass_diff', 'ugriz_z_ugriz_g_diff', 'ugriz_z_ugriz_u_diff', 'ugriz_z_dered_z_diff', 'ugriz_i_ugriz_r_diff', 'ugriz_i_dered_g_diff', 'ugriz_r_ugriz_g_diff', 'ugriz_r_dered_z_diff', 'ugriz_r_dered_r_diff','ugriz_r_dered_u_diff','ugriz_g_dered_u_diff', 'ugriz_u_dered_r_diff', 'ugriz_u_dered_g_diff', 'dered_z_dered_i_diff', 'dered_z_airmass_g_diff', 'dered_r_dered_g_diff', 'dered_r_dered_u_diff', 'dered_g_dered_u_diff', 'dered_g_airmass_i_diff', 'dered_g_airmass_g_diff', 'dered_u_airmass_i_diff','dered_airmass_sum_diff', 'ugriz_icolor', 'ugriz_ucolor',  'dered_orthogonal','class']

In [20]:
all_data = all_data[selected_columns].copy()

In [21]:
all_data.shape

(399923, 32)

In [22]:
all_data.describe()

,redshift,dered_r,dered_i,airmass_g,ugriz_r,ugriz_i,airmass_diff,ugriz_z_ugriz_g_diff,ugriz_z_ugriz_u_diff,ugriz_z_dered_z_diff,ugriz_i_ugriz_r_diff,ugriz_i_dered_g_diff,ugriz_r_ugriz_g_diff,ugriz_r_dered_z_diff,ugriz_r_dered_r_diff,ugriz_r_dered_u_diff,ugriz_g_dered_u_diff,ugriz_u_dered_r_diff,ugriz_u_dered_g_diff,dered_z_dered_i_diff,dered_z_airmass_g_diff,dered_r_dered_g_diff,dered_r_dered_u_diff,dered_g_dered_u_diff,dered_g_airmass_i_diff,dered_g_airmass_g_diff,dered_u_airmass_i_diff,dered_airmass_sum_diff,ugriz_icolor,ugriz_ucolor,dered_orthogonal,class
count,3.9992e+05,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,3.9992e+05,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000,399923.0000
mean,6.0401e-02,17.6165,17.2156,1.1766,17.7361,17.3043,6.7180e-03,-1.3548,-2.7759,0.1085,-0.4318,-0.9558,-0.6968,0.7665,0.1196,-1.8947,-1.1980,2.2375,1.5939,-0.2460,15.7930,-0.6436,-2.0143,-1.3708,17.0846,17.0835,18.4553,83.8133,0.3095,0.0825,0.0600,0.6931
std,3.0307e-01,1.4690,1.3264,0.1181,1.4626,1.3155,5.8439e-03,11.0723,11.1283,26.8449,0.5561,0.9324,0.6056,15.8518,0.3390,1.1223,0.8981,1.2037,0.9556,15.8369,15.8934,0.5412,1.0501,0.7285,1.6548,1.6549,1.9064,17.3982,0.5926,0.3428,0.5031,1.1828
min,-3.0149e+01,-8.7560,-3.6973,1.0001,-5.4387,5.3267,1.9000e-05,-36.3828,-54.5181,-62.2878,-30.6336,-14.9950,-79.9156,-27.9347,-30.1964,-27.6043,-72.2208,-19.8269,-21.4610,-10011.3000,-10000.4312,-15.2991,-16.4937,-13.0846,-19.8418,-19.8419,-31.8195,-9953.1315,-72.1962,-50.8115,-17.4669,-1.0000
25%,3.5279e-05,16.7882,16.4540,1.0885,16.8784,16.5243,1.8790e-03,-1.6109,-3.2235,0.0208,-0.4503,-1.1684,-0.8573,0.4068,0.0440,-2.2692,-1.4307,1.5842,1.1790,-0.2965,15.0579,-0.8122,-2.3608,-1.5538,16.1768,16.1754,17.3929,79.7350,0.1092,0.0021,-0.0837,0.0000
50%,4.7148e-02,17.4444,17.0881,1.1792,17.5267,17.1510,5.2690e-03,-1.2027,-2.4909,0.0406,-0.3604,-0.8504,-0.6261,0.6232,0.0702,-1.7264,-1.1142,1.9981,1.4040,-0.2012,15.7038,-0.5822,-1.8173,-1.2325,16.8568,16.8561,18.1003,83.0071,0.2500,0.0652,0.0094,1.0000
75%,9.4638e-02,18.2921,17.9075,1.2260,18.4289,18.0074,1.0346e-02,-0.7909,-1.8892,0.0699,-0.2425,-0.5282,-0.4276,0.8101,0.1137,-1.3181,-0.8945,2.5573,1.7574,-0.0845,16.4608,-0.3821,-1.4180,-1.0372,17.6996,17.6983,19.0151,87.2431,0.3752,0.1324,0.0569,2.0000
max,4.6390e+01,31.6536,30.9478,2.0797,46.6913,33.0259,5.8093e-02,6962.9626,6961.6056,16975.3922,24.6637,36.7597,63.2106,10011.7241,29.3532,51.2309,63.4692,40.4490,42.8546,13.6082,27.4969,14.8045,21.8777,14.5549,29.7876,29.7960,29.6671,131.0210,83.3487,59.3271,17.7349,2.0000


# 데이터셋 분리

In [23]:
# train set
X = all_data.loc[all_data['class'] != -1 , :]
X.drop(columns='class',inplace=True,axis=1)
y = all_data.loc[all_data['class'] != -1, 'class']
y.astype(int)

# test set
test = all_data.loc[all_data['class'] == -1, :]
test.drop(columns='class', inplace=True,axis=1)

# train set split
SEED = 2020
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state = SEED)

In [24]:
df_train.shape, df_test.shape, X.shape, y.shape, test.shape

((319923, 154), (80000, 153), (319923, 31), (319923,), (80000, 31))

In [25]:
X.describe()

,redshift,dered_r,dered_i,airmass_g,ugriz_r,ugriz_i,airmass_diff,ugriz_z_ugriz_g_diff,ugriz_z_ugriz_u_diff,ugriz_z_dered_z_diff,ugriz_i_ugriz_r_diff,ugriz_i_dered_g_diff,ugriz_r_ugriz_g_diff,ugriz_r_dered_z_diff,ugriz_r_dered_r_diff,ugriz_r_dered_u_diff,ugriz_g_dered_u_diff,ugriz_u_dered_r_diff,ugriz_u_dered_g_diff,dered_z_dered_i_diff,dered_z_airmass_g_diff,dered_r_dered_g_diff,dered_r_dered_u_diff,dered_g_dered_u_diff,dered_g_airmass_i_diff,dered_g_airmass_g_diff,dered_u_airmass_i_diff,dered_airmass_sum_diff,ugriz_icolor,ugriz_ucolor,dered_orthogonal
count,3.1992e+05,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,3.1992e+05,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000
mean,6.0481e-02,17.6158,17.2151,1.1765,17.7352,17.3037,6.7146e-03,-1.3715,-2.7921,0.0659,-0.4315,-0.9554,-0.6966,0.7408,0.1194,-1.8946,-1.1980,2.2367,1.5933,-0.2206,15.8179,-0.6434,-2.0140,-1.3707,17.0837,17.0826,18.4543,83.8351,0.3094,0.0826,0.0599
std,2.9110e-01,1.4672,1.3250,0.1181,1.4586,1.3141,5.8399e-03,1.1437,1.5900,0.2909,0.5494,0.9286,0.5761,0.8186,0.3230,1.1115,0.8763,1.1949,0.9451,0.4517,1.3186,0.5376,1.0457,0.7255,1.6536,1.6537,1.9046,7.1231,0.5612,0.3130,0.5009
min,-2.5915e+01,-8.7560,-3.6973,1.0001,7.7314,7.7115,1.9000e-05,-36.3828,-33.9750,-33.8940,-28.8183,-14.9950,-31.5613,-15.0466,-16.9807,-16.7362,-32.3721,-19.8269,-21.4610,-14.3327,-0.9700,-15.2991,-14.9929,-13.0846,-19.8418,-19.8419,-31.8195,-67.4564,-39.0770,-26.8336,-17.4669
25%,3.5724e-05,16.7874,16.4532,1.0885,16.8773,16.5240,1.8780e-03,-1.6103,-3.2230,0.0207,-0.4501,-1.1677,-0.8568,0.4064,0.0439,-2.2687,-1.4301,1.5835,1.1789,-0.2965,15.0579,-0.8119,-2.3602,-1.5530,16.1763,16.1749,17.3918,79.7333,0.1091,0.0020,-0.0839
50%,4.7153e-02,17.4434,17.0874,1.1792,17.5259,17.1498,5.2640e-03,-1.2016,-2.4892,0.0405,-0.3602,-0.8495,-0.6254,0.6228,0.0701,-1.7259,-1.1140,1.9973,1.4035,-0.2009,15.7030,-0.5815,-1.8169,-1.2325,16.8559,16.8550,18.0999,83.0003,0.2499,0.0651,0.0093
75%,9.4606e-02,18.2908,17.9070,1.2260,18.4279,18.0074,1.0346e-02,-0.7902,-1.8881,0.0699,-0.2423,-0.5278,-0.4273,0.8099,0.1136,-1.3176,-0.8945,2.5573,1.7566,-0.0845,16.4602,-0.3819,-1.4173,-1.0372,17.6978,17.6967,19.0150,87.2366,0.3752,0.1324,0.0568
max,4.4620e+01,31.2940,30.5509,2.0786,45.1299,32.8634,5.8064e-02,34.3737,29.8174,30.5841,15.6557,36.7574,34.9739,22.4823,29.3450,51.2227,54.5216,35.4717,42.7169,13.6082,27.4969,11.7109,21.8777,12.3501,29.5046,29.5111,29.6671,131.0210,22.8994,17.0388,16.9173


In [26]:
test.describe()

,redshift,dered_r,dered_i,airmass_g,ugriz_r,ugriz_i,airmass_diff,ugriz_z_ugriz_g_diff,ugriz_z_ugriz_u_diff,ugriz_z_dered_z_diff,ugriz_i_ugriz_r_diff,ugriz_i_dered_g_diff,ugriz_r_ugriz_g_diff,ugriz_r_dered_z_diff,ugriz_r_dered_r_diff,ugriz_r_dered_u_diff,ugriz_g_dered_u_diff,ugriz_u_dered_r_diff,ugriz_u_dered_g_diff,dered_z_dered_i_diff,dered_z_airmass_g_diff,dered_r_dered_g_diff,dered_r_dered_u_diff,dered_g_dered_u_diff,dered_g_airmass_i_diff,dered_g_airmass_g_diff,dered_u_airmass_i_diff,dered_airmass_sum_diff,ugriz_icolor,ugriz_ucolor,dered_orthogonal
count,8.0000e+04,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,8.0000e+04,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000
mean,6.0083e-02,17.6193,17.2178,1.1767,17.7397,17.3065,6.7318e-03,-1.2882,-2.7109,0.2787,-0.4333,-0.9573,-0.6974,0.8695,0.1204,-1.8952,-1.1978,2.2405,1.5961,-0.3476,15.6935,-0.6444,-2.0156,-1.3712,17.0882,17.0870,18.4593,83.7262,0.3100,0.0824,0.0604
std,3.4684e-01,1.4760,1.3324,0.1182,1.4786,1.3207,5.8600e-03,24.6502,24.6773,60.0184,0.5820,0.9475,0.7116,35.4045,0.3967,1.1647,0.9802,1.2383,0.9968,35.3975,35.4374,0.5555,1.0679,0.7405,1.6596,1.6596,1.9139,36.1981,0.7041,0.4422,0.5114
min,-3.0149e+01,-8.7560,-3.6973,1.0001,-5.4387,5.3267,2.1000e-05,-24.4105,-54.5181,-62.2878,-30.6336,-13.3576,-79.9156,-27.9347,-30.1964,-27.6043,-72.2208,-15.7468,-21.2096,-10011.3000,-10000.4312,-13.0229,-16.4937,-11.2314,-19.8418,-19.8419,-31.8195,-9953.1315,-72.1962,-50.8115,-14.3826
25%,3.3645e-05,16.7906,16.4562,1.0883,16.8820,16.5255,1.8810e-03,-1.6134,-3.2249,0.0210,-0.4508,-1.1714,-0.8593,0.4082,0.0441,-2.2711,-1.4330,1.5871,1.1794,-0.2968,15.0581,-0.8136,-2.3636,-1.5568,16.1781,16.1774,17.3975,79.7463,0.1099,0.0023,-0.0828
50%,4.7115e-02,17.4487,17.0912,1.1793,17.5299,17.1560,5.2950e-03,-1.2076,-2.4986,0.0406,-0.3613,-0.8540,-0.6289,0.6252,0.0704,-1.7289,-1.1151,2.0014,1.4058,-0.2023,15.7068,-0.5848,-1.8190,-1.2327,16.8606,16.8597,18.1019,83.0353,0.2504,0.0655,0.0097
75%,9.4769e-02,18.2952,17.9109,1.2262,18.4345,18.0080,1.0345e-02,-0.7938,-1.8930,0.0700,-0.2431,-0.5301,-0.4286,0.8105,0.1138,-1.3199,-0.8945,2.5575,1.7602,-0.0849,16.4633,-0.3828,-1.4208,-1.0377,17.7054,17.7039,19.0157,87.2768,0.3752,0.1323,0.0571
max,4.6390e+01,31.6536,30.9478,2.0797,46.6913,33.0259,5.8093e-02,6962.9626,6961.6056,16975.3922,24.6637,36.7597,63.2106,10011.7241,29.3532,51.2309,63.4692,40.4490,42.8546,12.4170,27.3017,14.8045,21.8777,14.5549,29.7876,29.7960,29.6429,123.8048,83.3487,59.3271,17.7349


# feature 선택

In [27]:
# Xgboost
xgb_params = {
    'n_jobs' : -1, 
    'n_estimators': 100,
    'eval_metric': 'mlogloss',
    'eta': 0.3, # learning_rate
    'booster': 'gbtree',
    'tree_method': 'auto',
    'objective': 'multi:softmax',
    'num_class': 3,
    'random_state': 2020
}

In [28]:
xgb_clf = xgb.XGBClassifier(**xgb_params)
xgb_clf.fit(X_train,y_train)
p = xgb_clf.predict(X_val)
print(accuracy_score(p,y_val))

0.9337862196151161


In [29]:
# xgb_clf.fit(X,y)
# p = xgb_clf.predict(test)

# submission = pd.read_csv('../input/dankook/sample_submission.csv')
# submission['class'] = p
# submission.to_csv('submission.csv',index=False, encoding='utf-8-sig')